In [14]:
import pandas as pd

plays_in = pd.read_csv("../Stage_3/Datasets/plays_in.csv")
team = pd.read_csv("../Stage_3/Datasets/team.csv")
games = pd.read_csv("../Stage_3/Datasets/Game.csv")

In [15]:
# Create map from abbrev to id
team_id_lookup = {
    abv : id for abv, id in zip(team["Team Abbrv"], team["Team_id"])
}

# Map Tm to id
ids = []
for tm in plays_in["Tm"]:
    ids.append(team_id_lookup[tm])

plays_in["TeamID"] = ids

# Drop Opp and Tm, add Tm_id (No longer need opp as we can infer it from game.csv)
# opp == team not corresponding to the team_id in plays_in
# we can also remove date since its in Game.csv
# we can also remove derived attributes:
# FG% = FG/FGA, 3P% = 3P/3PA, FT% = FT/FTA, PTS = 2*(FG - 3P) + 3*3P + FT,
# Res = W if teamId = homeId and home won or teamId = awayId and away won
# Res = L if teamId = homeId and home lost or teamId = awayId and away lost
derived_attributes = ["Opp", "PTS", "FT%", "3P%", "FG%", "Res"] 
plays_in = plays_in.drop(columns = ["Tm", "Date"] + derived_attributes)


In [16]:
# Reorder columns so PlayerID, GameID, and TeamID are first
first_cols = ["PlayerID", "GameID", "TeamID"]
cols = first_cols + [c for c in plays_in.columns if c not in first_cols]
plays_in = plays_in[cols]

In [17]:
# Sanity checks
print("All TeamIDs in plays_in exist in team table:",
      plays_in["TeamID"].isin(team["Team_id"]).all())

print("All GameIDs in plays_in exist in game table:",
      plays_in["GameID"].isin(games["GameID"]).all())

print("Missing TeamIDs:", plays_in["TeamID"].isna().sum())
print("Missing GameIDs:", plays_in["GameID"].isna().sum())
print("Duplicate (PlayerID, GameID) pairs:",
      plays_in.duplicated(subset=["PlayerID", "GameID"]).sum())

All TeamIDs in plays_in exist in team table: True
All GameIDs in plays_in exist in game table: True
Missing TeamIDs: 0
Missing GameIDs: 0
Duplicate (PlayerID, GameID) pairs: 0


In [18]:
# Save new csv
plays_in.to_csv("../Stage_3/Datasets/bcnf_plays_in.csv", index = False)